<a target="_blank" href="https://colab.research.google.com/github/giordamaug/HELP/blob/main/HELPpy/notebooks/prediction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://www.kaggle.com/notebooks/welcome?src=https://github.com/giordamaug/HELP/blob/main/HELPpy/notebooks/prediction.ipynb">
  <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/>
</a>

### 1. Install HELP from GitHub
Skip this cell if you already have installed HELP.

In [ ]:
!pip install git+https://github.com/giordamaug/HELP.git

### 2. Download the input files
In this cell we download from GitHub repository the label file and the attribute files. Skip this step if you already have these input files locally.

In [ ]:
tissue='Kidney'
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_HELP.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_BIO.csv
for i in range(5):
  !wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_CCcfs_{i}.csv
!wget https://raw.githubusercontent.com/giordamaug/HELP/main/data/{tissue}_EmbN2V_128.csv

### 3. Process the tissue attributes
In this code we load tissue gene attributes by several datafiles. We apply missing values fixing and data scaling with `sklearn.preprocessing.StandardScaler` on the `BIO` and `CCcfs` attributes, while no normalization and fixing on embedding attributes (`EmbN2V_128`). The attributes are all merged in one matrix by the `feature_assemble` function as input for the prediction model building.

In [6]:
%cd ../../data
tissue='Kidney'
import pandas as pd
from HELPpy.preprocess.loaders import feature_assemble_df
import os
df_y = pd.read_csv(f"{tissue}_HELP.csv", index_col=0)
df_y = df_y.replace({'aE': 'NE', 'sNE': 'NE'})    # E vs NE problem
#df_y = df_y[df_y['label'].isin(['aE']) == False]   # E vs sNE problem
#df_y = df_y[df_y['label'].isin(['E']) == False]    # aE vs sNE problem
#df_y = df_y[df_y['label'].isin(['sNE']) == False]  # E vs aE problem
print(df_y.value_counts(normalize=False))
features = [{'fname': f'{tissue}_BIO.csv', 'fixna' : False, 'normalize': 'std'},
            {'fname': f'{tissue}_CCcfs.csv', 'fixna' : False, 'normalize': 'std', 'nchunks' : 5},
            {'fname': f'{tissue}_EmbN2V_128.csv', 'fixna' : False, 'normalize': None}
            ]
df_X, df_y = feature_assemble_df(df_y, features=features, subsample=True, fold=1, verbose=True)
print(df_y.value_counts(normalize=False))
pd.merge(df_X, df_y, left_index=True, right_index=True, how='outer')

[Errno 2] No such file or directory: '../../data'
/Users/maurizio/HELP/data
label
NE       16678
E         1253
Name: count, dtype: int64
Majority NE 16678 minority E 1253
Subsampling with factor 1:1
[Kidney_BIO.csv] found 52532 Nan...
[Kidney_BIO.csv] Normalization with std ...


Loading file in chunks: 100%|██████████| 5/5 [00:02<00:00,  2.12it/s]


[Kidney_CCcfs.csv] found 6676644 Nan...
[Kidney_CCcfs.csv] Normalization with std ...
[Kidney_EmbN2V_128.csv] found 0 Nan...
[Kidney_EmbN2V_128.csv] No normalization...
2449 labeled genes over a total of 2506
(2449, 3456) data input
label
E        1242
NE       1207
Name: count, dtype: int64


,Gene length,Transcripts count,GC content,GTEX_kidney,Gene-Disease association,OncoDB_expression,HPA_kidney,GO-MF,GO-BP,GO-CC,...,Node2Vec_119,Node2Vec_120,Node2Vec_121,Node2Vec_122,Node2Vec_123,Node2Vec_124,Node2Vec_125,Node2Vec_126,Node2Vec_127,label
A2ML1,0.026017,0.041885,0.299948,5.123403e-07,0.017699,NaN,0.000000,0.069767,0.005128,0.038462,...,0.191344,-0.542462,0.746510,0.082089,-1.109212,0.406936,-1.332319,-0.363864,0.443284,NE
AAMP,0.002435,0.047120,0.512825,8.920776e-04,NaN,0.639246,0.001034,0.023256,0.015385,0.076923,...,0.229680,0.535889,0.432995,-0.131519,-0.017315,0.041874,-1.102342,-0.183920,0.005380,E
AARD,0.002528,0.005236,0.281277,4.451005e-06,NaN,0.651166,0.000018,0.000000,0.000000,NaN,...,-0.374058,0.275955,-0.530861,-0.406649,0.303176,-0.242258,-0.284188,-0.597559,0.314781,NE
AARS1,0.015241,0.214660,0.393300,NaN,0.002950,NaN,0.000805,0.186047,0.035897,0.076923,...,-0.001243,-0.385288,-0.113562,-0.308510,-0.278839,0.170950,0.012180,2.139750,-0.506839,E
AATF,0.043650,0.041885,0.235038,2.532015e-04,NaN,0.664865,0.000399,0.093023,0.056410,0.096154,...,0.170386,0.082064,-0.120705,0.654565,-1.567841,0.082283,0.605350,1.241107,0.709728,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZRSR2,0.016378,0.026178,0.232246,1.680569e-04,0.008850,0.687268,0.000214,0.093023,0.015385,0.057692,...,0.136922,-0.044823,-0.761108,0.168318,0.310074,-0.674975,-0.456626,-0.879874,-0.867850,NE
ZSCAN16,0.009043,0.005236,0.226139,8.095284e-05,0.000000,0.655000,0.000101,0.116279,0.000000,0.000000,...,-0.660592,0.147003,-0.706252,-0.476317,-0.129411,0.316885,-0.291835,-0.575228,-0.098236,NE
ZSCAN4,0.004120,0.000000,0.253359,1.217067e-06,NaN,0.663540,0.000000,0.116279,0.010256,0.038462,...,-1.004500,-0.126388,-0.656451,-0.844748,-0.043019,-1.335149,-0.127943,-0.745526,-0.472423,NE
ZSCAN5A,0.060112,0.089005,0.297505,1.449180e-05,0.000000,0.667713,0.000062,0.116279,0.000000,0.000000,...,-0.783969,-0.261120,-0.293825,1.969112,-0.169576,-0.272387,-0.411483,-0.451970,-1.590691,NE


### 4. Prediction
We process k-fold cross validation of a LightGBM classifier (`n_splits=5`), and then we store predictions and print metrics.

In [7]:
from HELPpy.models.prediction import predict_cv
df_scores, scores, predictions = predict_cv(df_X, df_y, n_splits=5, balanced=True, verbose=True)

{'E': 0, 'NE': 1}
label
E        1242
NE       1207
Name: count, dtype: int64
Classification with LGBM...


5-fold: 100%|██████████| 5/5 [00:14<00:00,  2.80s/it]


### 5. True Positive rates of context-specific EGs

In [9]:
df_scores

,measure
ROC-AUC,0.9459±0.0054
Accuracy,0.8730±0.0174
BA,0.8728±0.0174
Sensitivity,0.8897±0.0300
Specificity,0.8558±0.0273
MCC,0.7468±0.0351
CM,"[[1105, 137], [174, 1033]]"


In [10]:
import numpy as np
csEGs = pd.read_csv("csEG_Kidney.txt", index_col=0, header=None).index.values
indices = np.intersect1d(csEGs, predictions.index.values)
predictions = predictions.loc[indices]
num = len(predictions[predictions['label'] == predictions['prediction']])
den = len(predictions)
print(f"csEG Kidney TPR = {num /den:.3f} ({num}/{den})")

csEG Kidney TPR = 0.712 (42/59)
